In [1]:
# Danny Severe

In [2]:
# CREATE DATABASE iwdb;

In [3]:
# Connecting to the database

import pandas as pd
import psycopg2 as psy
import warnings

warnings.filterwarnings("ignore")

conn = psy.connect(database='iwdm', \
                   user='dannysevere', \
                   password='Severe21!', \
                   host='dbsevere.ct4e9f3blg2v.us-west-1.rds.amazonaws.com', \
                   port='5432')

cursor = conn.cursor()

In [4]:
# Creating the Customer Table

query = '''CREATE TABLE IF NOT EXISTS customer (
            CustID INT PRIMARY KEY,
            Fname VARCHAR(20) NOT NULL,
            Lname VARCHAR(20) NOT NULL,
            Age NUMERIC(2,0) NOT NULL,
            Cust_Type VARCHAR(6) NOT NULL,
            Cust_Since SMALLINT NOT NULL,
            Income INT NOT NULL,
            HH_Size SMALLINT NOT NULL,
            Color VARCHAR(6));'''

cursor.execute(query)
conn.commit()

In [5]:
# Creating the Game Table

query = '''CREATE TABLE IF NOT EXISTS GAME (
            GameID SERIAL PRIMARY KEY,
            HomeID SMALLINT NOT NULL,
            AwayID SMALLINT NOT NULL,
            FavoriteID SMALLINT NOT NULL,
            StadID SMALLINT,
            Score_Home SMALLINT NOT NULL,
            Score_Away SMALLINT NOT NULL,
            Over_Under NUMERIC(3,1) NOT NULL,
            Spread FLOAT NOT NULL,
            Winner_OU VARCHAR(5) NOT NULL,
            Winner_Line CHAR(4) NOT NULL,
            Date DATE NOT NULL,
            SeasonYr SMALLINT NOT NULL,
            Week VARCHAR(10) NOT NULL,
            Is_Playoff NUMERIC(1,0) NOT NULL,
            Neutral_Stad NUMERIC(1,0) NOT NULL,
            Temp SMALLINT,
            Wind_MPH SMALLINT,
            Humidity SMALLINT,
            Weather_Detail VARCHAR(40)
            );'''

cursor.execute(query)
conn.commit()

In [6]:
# Creating the Placed_Bets Table

query = '''CREATE TABLE IF NOT EXISTS placed_bets (
            BetID INT PRIMARY KEY,
            CustID INT NOT NULL,
            GameID INT NOT NULL,
            Bet_Amount SMALLINT NOT NULL,
            Bet_On VARCHAR(5) NOT NULL,
            Result VARCHAR(4) NOT NULL,
            Commission SMALLINT NOT NULL
            );'''

cursor.execute(query)
conn.commit()

In [7]:
# Creating the Stadium Table

query = '''CREATE TABLE IF NOT EXISTS stadium (
            StadID SERIAL PRIMARY KEY,
            StadName VARCHAR(40) NOT NULL,
            City VARCHAR(25),
            State VARCHAR(4),
            OpenYr SMALLINT,
            CloseYr SMALLINT,
            StadType VARCHAR(11),
            Address VARCHAR(75),
            Zip INT,
            Weather VARCHAR(8),
            Capacity INT,
            Surface VARCHAR(9),
            Station CHAR(11),
            Station_Name VARCHAR(75),
            Latitude FLOAT,
            Longitude FLOAT,
            Elevation FLOAT
            );'''

cursor.execute(query)
conn.commit()

In [8]:
# Creating the Team Table

query = '''CREATE TABLE IF NOT EXISTS team (
            TeamID SERIAL PRIMARY KEY,
            Team_Name VARCHAR(40) NOT NULL,
            Short_Name VARCHAR(20) NOT NULL,
            Team_Abrev CHAR(3) NOT NULL,
            Alt_Abrev CHAR(3) NOT NULL,
            Conference CHAR(3) NOT NULL,
            Division VARCHAR(7),
            Pre2002_Conf CHAR(3) NOT NULL,
            Pre2002_Div VARCHAR(7)
            );'''

cursor.execute(query)
conn.commit()

In [9]:
# Loading in the SQL Tables
x = 1

if x == 0:
    import pymssql

    conn = pymssql.connect(
         host='stairwaytoheaven.usu.edu', # Server name goes in quotes.
         user='5330user', # Username goes in quotes.
         password='pipelinesnow', # Password goes in quotes
         database='ironwill') # Database to use goes in quotes

    cursor = conn.cursor()

    query = ('''SELECT *
                FROM customer_table; ''')

    df = pd.read_sql(query, conn)

    query = ('''SELECT *
                FROM betlog; ''')

    df2 = pd.read_sql(query, conn)

    conn.close()

In [10]:
# Populating the Customer Table

conn = psy.connect(database='iwdm', \
                   user='dannysevere', \
                   password='Severe21!', \
                   host='dbsevere.ct4e9f3blg2v.us-west-1.rds.amazonaws.com', \
                   port='5432')

cursor = conn.cursor()

exception = []
z = 0

while z == 0:
    cursor.execute("""SELECT EXISTS(
                        SELECT 1
                        FROM customer
                        WHERE custid = 1)"""
                        )
    y = cursor.fetchone()
        
    if y[0] == True: 
        z = 1

    else:

        for x in df.index:


            try:
                parts = df['customer_name'].loc[x].split()  #splitting the name column into their component parts
                fname = parts[0]
                lname = parts[1]
            except:
                print('Name Error') #error tracking
                exception.append(x)

            try:
                age = int(df['customer_age'].loc[x])  #Checking if age is numeric
            except:
                print('Age Error')
                exception.append(x)

            if df['customer_type'].loc[x] not in ('online', 'phone', 'local'): # Checking if there are invalid entries
                print('Type Error')
                exception.append(x)
            else:
                ctype = df['customer_type'].loc[x]

            try:
                since = int(df['customer_since'].loc[x])  # Checking if customer_since is numeric
            except:
                print('Since Error')
                exception.append(x)

            try:
                income = int(df['customer_income'].loc[x]) # Checking if income is numeric
            except:
                print('Income Error')
                exception.append(x) 

            try:
                size = int(df['household_size'].loc[x]) # Checking if household_size is numeric
            except:
                print('Size Error')
                exception.append(x) 

            if df['mode_color'].loc[x] not in ('red', 'yellow', 'blue', 'orange', 'green', 'purple', 'black', 'white'):
                print('Color Error')
                exception.append(x)
            else:
                color = df['mode_color'].loc[x] # Validating Color entry

            try:
                ID = int(df['customer_id'].loc[x]) # Checking if custid is numeric
            except:
                print('ID Error')
                exception.append(x)

            try:
                cursor.execute('''INSERT INTO customer (CustID, Fname, Lname, Age, Cust_Type, Cust_Since, Income, HH_Size, Color)
                                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                                 (ID, fname, lname, age, ctype, since, income, size, color))
                conn.commit()
            except Exception as e:
                print('Error:', e)
                exception.append(x)


In [11]:
# importing the Team Table
import os

df = pd.read_csv('/Users/dannysevere/Documents/python/nfl_teams.csv')
print(df.head())

           team_name team_name_short team_id team_id_pfr team_conference  \
0  Arizona Cardinals       Cardinals     ARI         CRD             NFC   
1    Atlanta Falcons         Falcons     ATL         ATL             NFC   
2    Baltimore Colts           Colts     IND         CLT             AFC   
3   Baltimore Ravens          Ravens     BAL         RAV             AFC   
4    Boston Patriots        Patriots      NE         NWE             AFC   

  team_division team_conference_pre2002 team_division_pre2002  Unnamed: 8  \
0      NFC West                     NFC              NFC West         NaN   
1     NFC South                     NFC              NFC West         NaN   
2           NaN                     AFC              AFC East         NaN   
3     AFC North                     AFC           AFC Central         NaN   
4           NaN                     AFC                   NaN         NaN   

  Unnamed: 9  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4    

In [12]:
# Populating the Team Table
z = 0

while z == 0:
    cursor.execute("""SELECT EXISTS(
                        SELECT 1
                        FROM team
                        WHERE teamid = 1)"""
                        )
    y = cursor.fetchone()
        
    if y[0] == True: 
        z = 1

    else:
        for x in df.index:

            if pd.isnull(df['team_division'].loc[x]):
                div = None

            else:
                parts = df['team_division'].loc[x].split()  #extracting the divsion from conference/division input
                div = parts[1]

            if pd.isnull(df['team_division_pre2002'].loc[x]):
                div02 = None

            else:
                part = df['team_division_pre2002'].loc[x].split()
                div02 = part[1]

            cursor.execute('''INSERT INTO team (Team_Name, Short_Name, Team_Abrev, Alt_Abrev, Conference, Division, Pre2002_Conf, Pre2002_Div)
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''',
                          (df['team_name'].loc[x], df['team_name_short'].loc[x], df['team_id'].loc[x], df['team_id_pfr'].loc[x], df['team_conference'].loc[x], div, df['team_conference_pre2002'].loc[x], div02))
            conn.commit()


In [13]:
# importing the stadium data

df = pd.read_csv('/Users/dannysevere/Documents/python/nfl_stadiums.csv', encoding='latin1')
df = df.rename(columns={'ï»¿stadium_name': 'stadium_name'})
print(df)

             stadium_name  stadium_location  stadium_open  stadium_close  \
0        Acrisure Stadium    Pittsburgh, PA        2001.0            NaN   
1              Alamo Dome   San Antonio, TX           NaN            NaN   
2       Allegiant Stadium      Paradise, NV        2020.0            NaN   
3           Allianz Arena   Munich, Germany           NaN            NaN   
4          Alltel Stadium  Jacksonville, FL           NaN            NaN   
..                    ...               ...           ...            ...   
113  War Memorial Stadium       Buffalo, NY        1960.0         1972.0   
114       Wembley Stadium        London, UK        2007.0            NaN   
115         Wrigley Field       Chicago, IL        1920.0         1970.0   
116             Yale Bowl     New Haven, CT           NaN            NaN   
117        Yankee Stadium         Bronx, NY           NaN            NaN   

    stadium_type                              stadium_address  \
0        outdoor     1

In [14]:
# Populating the Stadium Table
z = 0

while z == 0:
    cursor.execute("""SELECT EXISTS(
                        SELECT 1
                        FROM stadium
                        WHERE stadid = 1)"""
                        )
    y = cursor.fetchone()
        
    if y[0] == True: 
        z = 1

    else:
        for x in df.index:
            stadname = df['stadium_name'].loc[x].replace("'", "").replace("-", "").replace(".", "") 

            if pd.isnull(df['stadium_location'].loc[x]): #Splitting Location into City and State columns
                city = None
                state = None
            else:
                city, state = df['stadium_location'].loc[x].split(", ")
                if len(state) > 2:
                    state = state[:3].upper()

            try:
                if pd.isnull(df['stadium_open'].loc[x]): 
                    openyr = None
                else:
                    openyr = int(df['stadium_open'].loc[x])
            except:
                print('year error')

            try:
                if pd.isnull(df['stadium_close'].loc[x]):
                    closeyr = None
                else:
                    closeyr = int(df['stadium_close'].loc[x])
            except:
                print('year error')

            try:
                if pd.isnull(df['stadium_type'].loc[x]):
                    stadtype = None
                else:
                    stadtype = df['stadium_type'].loc[x]
            except:
                print('stadtype error')

            try:
                if pd.isnull(df['stadium_address'].loc[x]):     # splitting the address column and substituting zip for weather code to avoid redundancy
                    address = None
                    zipcode = None
                else:
                    addressparts = df['stadium_address'].loc[x].split(", ")
                    address = addressparts[0]
                    zipcode = addressparts[-1].split()[1]
                    if len(zipcode) > 5 or len(zipcode) < 5:
                        zipcode = None
            except:
                zipcode = None

            if pd.isnull(df['weather'].loc[x]):
                 weather = None
            else:
                weather = df['weather'].loc[x]

            if pd.isnull(df['stadium_capacity'].loc[x]):
                    capacity = None
            else:
                capacity = int(df['stadium_capacity'].loc[x].replace(',', ''))

            if pd.isnull(df['stadium_surface'].loc[x]):
                    surface = None
            else:
                surface_value = df['stadium_surface'].loc[x]  # Reducing entries to grass and fieldturf to avoid confusion

                if 'turf' in surface_value.lower():
                    surface = 'fieldturf'
                elif 'grass' in surface_value.lower():
                    surface = 'grass'

            if pd.isnull(df['STATION'].loc[x]):
                    station = None
            else:
                station = df['STATION'].loc[x]

            if pd.isnull(df['NAME'].loc[x]):
                    station_name = None
            else:
                station_name = df['NAME'].loc[x]

            if pd.isnull(df['LATITUDE'].loc[x]):
                    latitude = None
            else:
                latitude  = float(df['LATITUDE'].loc[x])

            if pd.isnull(df['LONGITUDE'].loc[x]):
                    longitude = None
            else:
                longitude = float(df['LONGITUDE'].loc[x])

            if pd.isnull(df['ELEVATION'].loc[x]):
                    elevation = None
            else:
                elevation = float(df['ELEVATION'].loc[x])

            cursor.execute('''INSERT INTO stadium (StadName, City, State, OpenYr, CloseYr, StadType, Address, Zip, Weather, Capacity, Surface, Station, Station_Name, Latitude, Longitude, Elevation)
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                          (stadname, city, state, openyr, closeyr, stadtype, address, zipcode, weather, capacity, surface, station, station_name, latitude, longitude, elevation))
            conn.commit()

    

In [15]:
# importing Game Data

df = pd.read_csv('/Users/dannysevere/Documents/python/spread_scores-1.csv')
print(df)

     schedule_date  schedule_season schedule_week  schedule_playoff  \
0           9/3/00             2000             1             False   
1           9/3/00             2000             1             False   
2           9/3/00             2000             1             False   
3           9/3/00             2000             1             False   
4           9/3/00             2000             1             False   
...            ...              ...           ...               ...   
6157       1/22/23             2022      Division              True   
6158       1/22/23             2022      Division              True   
6159       1/29/23             2022    Conference              True   
6160       1/29/23             2022    Conference              True   
6161       2/12/23             2022     Superbowl              True   

                team_home  score_home  score_away             team_away  \
0         Atlanta Falcons          36          28   San Francisco 49ers 

In [16]:
# Populating Game Table
exception = []
z = 0

while z == 0:
    cursor.execute("""SELECT EXISTS(
                        SELECT 1
                        FROM game
                        WHERE gameid = 1)"""
                        )
    y = cursor.fetchone()
        
    if y[0] == True: 
        z = 1

    else:
        for x in df.index:

            # filtering data to include only games after 2015
            yr = int(df['schedule_season'].loc[x])

            if yr >= 2015:
                try:
                    # preparing the home team variable
                    home_name = df['team_home'].loc[x]
                    query = '''SELECT teamid
                                FROM team
                                WHERE team_name = '%s';
                            '''% (home_name)

                    df1 = pd.read_sql(query, conn)
                    home = int(df1.iloc[0,0])

                    # preparing the away team variable
                    away_name = df['team_away'].loc[x]
                    query = '''SELECT teamid
                                FROM team
                                WHERE team_name = '%s';
                            '''% (away_name)

                    df1 = pd.read_sql(query, conn)
                    away = int(df1.iloc[0,0])

                    # preparing the favorite variable
                    favid = df['team_favorite_id'].loc[x]

                    if favid == 'PICK':
                        fav = 0
                    else:
                        query = '''SELECT teamid
                                    FROM team
                                    WHERE team_abrev = '%s' OR alt_abrev = '%s';
                                '''% (favid, favid)

                        df1 = pd.read_sql(query, conn)
                        fav = int(df1.iloc[0,0])

                    # preparing the stadium variable

                    stad = df['stadium'].loc[x].replace("'", "").replace("-", "").replace(".", "")
                    query = '''SELECT stadid 
                                FROM stadium 
                                WHERE stadname = '%s';''' % (stad)

                    df1 = pd.read_sql(query, conn)

                    if not df1.empty:  # Check if the DataFrame is not empty
                        stadid = int(df1.iloc[0, 0])
                    else:
                        print("Stadium not found:", stad)
                        stadid = None

                    # home and away score variables
                    scoreh = int(df['score_home'].loc[x])
                    scorea = int(df['score_away'].loc[x])

                    # Over Under variable
                    ou = float(df['over_under_line'].loc[x])

                    # Spread
                    sp = float(df['spread_favorite'].loc[x])

                    # calculating the winner for the over/under bet
                    tot = scoreh + scorea
                    if tot > ou:
                        ou_win = 'over'
                    elif tot < ou:
                        ou_win = 'under'
                    else:
                        ou_win = 'push'

                    # calculating winner of the spread bet
                    if scoreh > scorea and fav == home:
                        if scoreh + sp > scorea:
                            win_sp = 'home'
                        elif scoreh + sp < scorea:
                            win_sp = 'away'
                        else:
                            win_sp = 'push'

                    elif scoreh < scorea and fav == away:
                        if scorea + sp > scoreh:
                            win_sp = 'away'
                        elif scorea + sp < scoreh:
                            win_sp = 'home'
                        else:
                            win_sp = 'push'
                    elif scoreh < scorea and fav == home:
                        win_sp = 'away'

                    elif scoreh > scorea and fav == away:
                        win_sp = 'home'

                    # Date variable
                    from datetime import datetime
                    dates = df['schedule_date'].loc[x]
                    date = datetime.strptime(dates, '%m/%d/%y')
                    date = date.date()

                    # season year
                    yr = int(df['schedule_season'].loc[x])

                    # week of season
                    week = df['schedule_week'].loc[x]

                    # Playoff = 1
                    playoff = 1 if df['schedule_playoff'].loc[x] else 0

                    # Neutral Stadium
                    neut = 1 if df['stadium_neutral'].loc[x] else 0

                    # Temp
                    if pd.isnull(df['weather_temperature'].loc[x]):
                        temp = None
                    else:
                        temp = int(df['weather_temperature'].loc[x])

                    # Wind
                    if pd.isnull(df['weather_wind_mph'].loc[x]):
                        wind = None
                    else:
                        wind = int(df['weather_wind_mph'].loc[x])

                    # Humidity
                    if pd.isnull(df['humidity'].loc[x]):
                        humid = None
                    else:
                        humid = int(df['humidity'].loc[x])

                    # Weather Detail
                    if pd.isnull(df['weather_detail'].loc[x]):
                        detail = None
                    else:
                        detail = df['weather_detail'].loc[x]

                    cursor.execute('''INSERT INTO game (HomeID, AwayID, FavoriteID, StadID, Score_Home, Score_Away, Over_Under, Spread,
                                                        Winner_OU, Winner_Line, Date, SeasonYr, Week, Is_Playoff, Neutral_Stad,
                                                        Temp, Wind_MPH, Humidity, Weather_Detail)
                                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                                  (home, away, fav, stadid, scoreh, scorea, ou, sp, ou_win, win_sp, date, yr, week, playoff, neut, temp, wind, humid, detail))
                    conn.commit()
                except:
                    print('Error', df.loc(x))
                    exception.append(df.loc(x))

In [17]:
# viewing the bet_log data
#df = df2
#print(df)

In [20]:
exception = []
last_game = None
last_bet_on = None
z = 1
s = 0
d = 0
while z == 0:
    for x in df.index:
        # storing bet_id in a variable
        betid = int(df['bet_id'].loc[x])

        cursor.execute("""SELECT EXISTS(
                            SELECT 1
                            FROM placed_bets
                            WHERE betid = '%s')""" % (betid))
        y = cursor.fetchone()

        if y[0] == True:
            s += 1
            d += 1
            if s == 1000:
                print(d, 'records have been processed')
                s = 0
        else:
            # storing bet_id in a variable
            betid = int(df['bet_id'].loc[x])

            # storing customer_id in a variable
            custid = int(df['customer_id'].loc[x])

            try:
                # handling and linking bet_id to the correct table
                game = df['game_id'].loc[x].split('-')
                #print(game)
                home = game[1].strip()
                #print(home)
                away = game[2].strip()
                #print(away)
                year = int(game[0][:4])
                #print(year)

                if game[0][4:] == '19':
                    week = 'Wildcard'
                elif game[0][4:] == '20':
                    week = 'Division'
                elif game[0][4:] == '21':
                    week = 'Conference'
                elif game[0][4:] == '22':
                    week = 'Superbowl'
                else:
                    week = int(game[0][4:])  
                #print(week)

                if home == 'JAC' or home == 'JAC ':
                    homeid = 18
                elif home == 'LV' or home == 'LV ' or home == 'LVR' or home == 'LVR ':
                    homeid = 20
                elif home == 'ARI' or home == 'ARI ':
                    homeid = 1
                elif home == 'NE' or home == 'NE ':
                    homeid = 26
                elif home == 'IND' or home == 'IND ':
                    homeid = 17
                elif home == 'IND' or home == 'IND ':
                    homeid = 17
                elif home == 'TEN' or home == 'TEN ':
                    homeid = 41
                elif home == 'LAC' or home == 'LAC ':
                    homeid = 21
                elif home == 'LAR' or home == 'LAR ':
                    homeid = 23
                elif home == 'WAS' or home == 'WAS ':
                    homeid = 42
                else:
                    query = '''SELECT teamid
                            FROM team
                            WHERE team_abrev = '%s' OR alt_abrev = '%s'; 
                        '''% (home, home)

                    df1 = pd.read_sql(query, conn) 
                    if not df1.empty:
                        homeid = int(df1.iloc[0,0])
                #print(homeid)
                                            # finding the gameid based on the variables identified above
                query = '''SELECT gameid
                            FROM game
                            WHERE seasonyr = '%s' AND week = '%s' AND homeid = '%s';
                        '''% (year, week, homeid)

                df1 = pd.read_sql(query, conn)
                if not df1.empty:
                        gameid = int(df1.iloc[0,0])
                else:
                    print('no data found for game')
            except:
                print('game error')
                game = df['game_id'].loc[x]
                if last_game is not None:
                    if last_game != game:
                        exception.append(df.iloc[x].to_string(index=False))
                        last_game = game
                else:
                    last_game = game
                    exception.append(df.iloc[x].to_string(index=False))

            # storing the bet amount
            amount = int(df['bet_amount'].loc[x])

            # Handling the bet_on column
            try:
                bet_on_team = df['bet_on'].loc[x]
                if bet_on_team == 'over' or bet_on_team == 'under' or bet_on_team == 'push':
                    bet_on = bet_on_team.strip()
                else:
                    query = '''SELECT team_abrev
                                FROM team
                                WHERE team_name = '%s';
                        '''% (bet_on_team)

                    df1 = pd.read_sql(query, conn)
                    if not df1.empty:
                        bet_on = df1.iloc[0,0].strip()
                    else:
                        print('error with bet on team', bet_on_team)
            except:
                print('error with bet_on')

            # Calculating Result Variable
            try:
                query = '''SELECT winner_ou
                                FROM game
                                WHERE gameid = '%s';
                        '''% (gameid)                 # pulling the over/under result into a variable

                df1 = pd.read_sql(query, conn)
                if not df1.empty:
                        win_ou = df1.iloc[0,0]
                else:
                    print("No data found for result:")


                query = '''SELECT winner_line
                                FROM game
                                WHERE gameid = '%s';
                        '''% (gameid)                     # pulling result of the spread into a variable for comparison

                df1 = pd.read_sql(query, conn)
                if not df1.empty:
                        win_line = df1.iloc[0,0]
                else:
                    print("No data found for result:")


                if bet_on == 'over' or bet_on == 'under' or bet_on == 'push':
                    if bet_on == 'over' and win_ou == 'over':
                        result = 'win'
                    elif bet_on == 'over' and win_ou == 'under':  # comparing bet_on to result to determine if bet is a win or loss
                        result = 'loss'
                    elif bet_on == 'under' and win_ou == 'under':
                        result = 'win'
                    elif bet_on == 'under' and win_ou == 'over':
                        result = 'loss'
                    elif bet_on == 'push' and win_ou == 'push':
                        result = 'win'
                    elif bet_on == 'push' and win_ou == 'over':
                        result = 'loss'
                    elif bet_on == 'push' and win_ou == 'under':
                        result = 'loss'
                    elif (win_ou == 'push' or win_line == 'push') and (bet_on == 'under' or bet_on == 'over'):
                        result = 'push'
                else:
                    query = '''SELECT teamid
                                FROM team
                                WHERE team_abrev = '%s' OR alt_abrev = '%s';
                        '''% (bet_on, bet_on)

                    df1 = pd.read_sql(query, conn)
                    if not df1.empty:
                        bet_on_id = int(df1.iloc[0,0])        # converting team for spread bet into team_id and comparing with home or away id to determine result
                    else:
                        print("No data found for result:")

                    query = '''SELECT homeid
                                FROM game
                                WHERE gameid = '%s';
                        '''% (gameid)

                    df1 = pd.read_sql(query, conn)
                    if not df1.empty:
                        home_id = int(df1.iloc[0,0])
                    else:
                        print("No data found for result:")

                    query = '''SELECT awayid
                                FROM game
                                WHERE gameid = '%s';
                        '''% (gameid)

                    df1 = pd.read_sql(query, conn)
                    if not df1.empty:
                        away_id = int(df1.iloc[0,0])
                    else:
                        print("No data found for result:")

                    if win_line == 'home' and bet_on_id == home_id:
                        result = 'win'
                    elif win_line == 'home' and bet_on_id == away_id:
                        result = 'loss'
                    if win_line == 'away' and bet_on_id == home_id:
                        result = 'loss'
                    elif win_line == 'away' and bet_on_id == away_id:
                        result = 'win'
                    elif win_line == 'push':
                        result = 'push'
            except:
                print('result error', bet_on, bet_on_team)


            # Calculating commission
            amount = int(df['bet_amount'].loc[x])

            if amount <= 1000:
                com = int(amount * .1)           # calculating bets under 1000

            elif amount > 1000 and amount <= 5000:
                k2 = amount - 1000
                k1 = amount - k2
                k1 = int(k1 * .1)                # calculating bets under 5k
                k2 = int(k2 * .08)
                com = int(k1 + k2)

            elif amount > 5000 and amount <= 20000:
                k3 = amount - 5000
                k2 = (amount - k3) - 1000
                k1 = 1000
                k1 = int(k1 * .1)               # calculating bets over 5k
                k2 = int(k2 * .08)
                k3 = int(k3 * .06)
                com = int(k1 + k2 + k3)
            #print(game, year, week, home, homeid, away)
            try:
                cursor.execute('''INSERT INTO placed_bets (BetID, CustID, GameID, Bet_Amount, Bet_On, Result, Commission)
                                              VALUES (%s, %s, %s, %s, %s, %s, %s)''',
                                      (betid, custid, gameid, amount, bet_on, result, com))
                conn.commit()
            except:
                print('Ingestion Error', df.iloc(x))
                exception.append(df.iloc[x].to_string(index=False))
                conn.rollback()


            s += 1
            d += 1
            if s == 50:
                print(x+1, 'records have been processed')
                print('Records that couldnt be ingested:', exception, '\n')
                s = 0
            elif x == len(df.index)-1:
                print('Ingestion complete')
    z = 1
            


In [21]:
# Test Queries
# 1

query = ('''SELECT *
            FROM customer
            LIMIT 5;
            ''')
query2 = ('''SELECT *
            FROM team
            LIMIT 5;
            ''')
query3 =    ('''SELECT *
            FROM stadium
            LIMIT 5;
            ''')
query4 =    ('''SELECT *
            FROM game
            LIMIT 5;
            ''')
query5 =        ('''SELECT *
            FROM placed_bets
            LIMIT 5;
            ''')
df = pd.read_sql(query, conn)
df2 = pd.read_sql(query2, conn)
df3 = pd.read_sql(query3, conn)
df4 = pd.read_sql(query4, conn)
df5 = pd.read_sql(query5, conn)
print('-------------------------------------------------------------------------------------------------------------\n','Customer Table', df, '\n-------------------------------------------------------------------------------------------------------------')
print('Team Table', df2, '\n-------------------------------------------------------------------------------------------------------------')
print('Stadium Table', df3, '\n-------------------------------------------------------------------------------------------------------------')
print('Game Table', df4, '\n-------------------------------------------------------------------------------------------------------------')
print('Placed_Bets Table', df5, '\n-------------------------------------------------------------------------------------------------------------')

-------------------------------------------------------------------------------------------------------------
 Customer Table    custid     fname    lname   age cust_type  cust_since  income  hh_size  \
0       1   Leilani    Garza  31.0    online        2019   90000        4   
1       2  Samantha   Curtis  45.0     phone        2020  102000        1   
2       3   Matthew  Morales  31.0     local        2021  113000        1   
3       4    Holden  Randall  34.0    online        2022  260000        3   
4       5     Rylan   Oliver  27.0     local        2018   65000        1   

    color  
0   green  
1  purple  
2  orange  
3     red  
4    blue   
-------------------------------------------------------------------------------------------------------------
Team Table    teamid          team_name short_name team_abrev alt_abrev conference  \
0       1  Arizona Cardinals  Cardinals        ARI       CRD        NFC   
1       2    Atlanta Falcons    Falcons        ATL       ATL       

In [22]:
# 2
query = ('''
            WITH ct AS (SELECT custid, SUM(commission) AS total_commission
                    FROM placed_bets
                    GROUP BY custid
                    HAVING SUM(commission) > 20000),
            ct2 AS (SELECT COUNT(*) AS total_customers
                    FROM customer),
            ct3 AS (SELECT COUNT(*) AS high_commission_customers, ct2.total_customers
                    FROM ct CROSS JOIN ct2
                    GROUP BY ct2.total_customers)
            
            SELECT high_commission_customers, total_customers,
                    CAST(high_commission_customers * 100.0 / total_customers AS INT) AS percent_high_commission
            FROM ct3;
            ''')

df = pd.read_sql(query, conn)
print('-------------------------------------------------------------------------------------------------------------\n','Customer Table', df, '\n-------------------------------------------------------------------------------------------------------------')

query = ('''
            WITH ct AS (SELECT custid, SUM(commission) AS total_commission
                    FROM placed_bets
                    GROUP BY custid
                    HAVING SUM(commission) > 20000)
            
            SELECT fname, lname, total_commission
            FROM customer c JOIN ct ON c.custid = ct.custid
            ORDER BY total_commission DESC
            LIMIT 20;
            ''')

df1 = pd.read_sql(query, conn)
print(df1) 

-------------------------------------------------------------------------------------------------------------
 Customer Table    high_commission_customers  total_customers  percent_high_commission
0                        240             2000                       12 
-------------------------------------------------------------------------------------------------------------
      fname        lname  total_commission
0      Neal         lark            188940
1      Zoie  Blankenship            177866
2    Chanel       Martin            135865
3    Gunner       Morris            134179
4     Amari     Chandler            134142
5   Jackson        White            132251
6     Jacob     Gonzalez            131242
7    Xander        Jones            124660
8     Miles        Young            121320
9    Steven        Brown            118912
10   Xavier         Lowe            118042
11   Harvey        Ayers            115411
12   Harris     Campbell            114117
13  Eleanor       T

In [23]:
# 3

query = ('''
            WITH ct AS (SELECT c.custid, c.fname, c.lname, COUNT(p.betid) AS bet_count
                        FROM customer c JOIN placed_bets p ON c.custid = p.custid
                        GROUP BY c.custid, c.fname, c.lname
                        HAVING COUNT(betid) >= 6),
                        
            ct2 AS (SELECT c.custid, COUNT(p.betid) AS win_count, SUM(p.bet_amount) as total_bet
                    FROM customer c JOIN placed_bets p ON c.custid = p.custid
                    WHERE p.result = 'win'
                    GROUP BY c.custid
                    HAVING COUNT(p.betid) >= 6)
            
            SELECT ct.fname, ct.lname, ct.bet_count, ct2.win_count, CAST(ct2.win_count * 100 / ct.bet_count AS INT) AS win_percentage, (ct2.total_bet * 2) AS total_won
            FROM ct JOIN ct2 ON ct.custid = ct2.custid
            ORDER BY win_percentage DESC, total_won DESC;
            ''')

df = pd.read_sql(query, conn)
print(df) 

        fname       lname  bet_count  win_count  win_percentage  total_won
0       Layla      Norton          9          9             100      54100
1       Brian       Price         16         15              93       5550
2     Spencer  Fitzgerald         13         12              92       2750
3     Waverly     Shields          9          8              88      31800
4       Anika    Ferguson          8          7              87       1900
...       ...         ...        ...        ...             ...        ...
1605     Emmy     Kennedy         67         14              20     115100
1606     Owen        Long        111         23              20      18900
1607    Jamie       Green         74         15              20      15950
1608    Grace       Jones        135         26              19      18700
1609     Aria      Morris         31          6              19       4900

[1610 rows x 6 columns]


In [24]:
#4

query = ('''
            WITH win AS (SELECT custid, COUNT(betid) AS win_count, SUM(bet_amount)*2 as total_won, SUM(commission) as total_commission_win
                        FROM placed_bets
                        WHERE result = 'win'
                        GROUP BY custid),
                        
            loss AS (SELECT custid, COUNT(betid) AS loss_count, SUM(bet_amount) as total_loss, SUM(commission) as total_commission_loss
                        FROM placed_bets
                        WHERE result = 'loss'
                        GROUP BY custid),
                        
            value AS (SELECT w.custid, w.win_count, (w.win_count + l.loss_count) AS bet_count, (w.total_won - w.total_commission_win) AS win_value,
                                        (l.total_loss + l.total_commission_loss) AS loss_value
                        FROM win w JOIN loss l ON w.custid = l.custid
                        )
            
            SELECT c.fname, c.lname, v.bet_count, v.win_count, ( v.win_value - v.loss_value) AS net_loss
            FROM customer c JOIN value v ON c.custid = v.custid
            ORDER BY net_loss DESC
            LIMIT 20;
            ''')

df = pd.read_sql(query, conn)
print(df) 

        fname        lname  bet_count  win_count  net_loss
0        Zoie  Blankenship        163         98   2000794
1   Alexander      Gilbert         75         50   1126366
2     Eleanor       Torres        171        101    967620
3        Elsa        Henry        142         78    932886
4      Steven        Brown        175         95    847538
5       Jacob     Gonzalez        182         95    827668
6      Harris     Campbell        166         79    821413
7       Amari     Chandler        188        102    760858
8       Jacob        Scott        158         98    655824
9       Miles        Young        256        116    616865
10   Benjamin      Sanchez        189        111    615999
11      Ariah      O'Brien        126         58    590134
12    Delilah      Nichols        184         95    560055
13      Logan         Hill         63         37    527762
14   Madelynn        Scott        165         89    486656
15  Christian       Melton        129         67    4865

In [37]:
#5

query = ('''WITH win AS (SELECT g.gameid, bet_amount
                            FROM placed_bets p JOIN game g ON g.gameid = p.gameid
                            WHERE result = 'loss' and seasonyr = 2022),
            loss AS (SELECT g.gameid, bet_amount
                            FROM placed_bets p JOIN game g ON g.gameid = p.gameid
                            WHERE result = 'win' and seasonyr = 2022),
            wk AS (SELECT CASE
                        WHEN week = 'Wildcard' THEN 19
                        WHEN week = 'Division' THEN 20
                        WHEN week = 'Conference' THEN 21
                        WHEN week = 'Superbowl' THEN 22
                        ELSE CAST(week AS INTEGER)
                    END AS week, g.gameid, SUM(w.bet_amount) AS book_win, SUM(l.bet_amount) AS book_lose
            from game g JOIN win w ON w.gameid = g.gameid JOIN loss l ON g.gameid = l.gameid
            WHERE seasonyr = 2022
            GROUP BY week, g.gameid
            ORDER BY week),
            
            ct AS (SELECT week, CASE
                    WHEN book_win >= book_lose THEN 1
                    ELSE 0
                    END AS win, CASE
                                    WHEN book_win < book_lose THEN 1
                                    ELSE 0
                                    END as lose
            FROM wk)
            
            SELECT week, COUNT(*) as games_played, SUM(win) as games_won, SUM(lose) as games_lost, ROUND((SUM(win) * 100.0 / COUNT(*)), 0) AS win_percentage,
                    ROUND((SUM(lose) * 100.0 / COUNT(*)), 0) AS lose_percentage
            FROM ct
            GROUP by week
        
            ''')
df = pd.read_sql(query, conn)

print(df) 

    week  games_played  games_won  games_lost  win_percentage  lose_percentage
0      1            16          7           9            44.0             56.0
1      2            16          6          10            38.0             63.0
2      3            16         10           6            63.0             38.0
3      4            16         10           6            63.0             38.0
4      5            16          9           7            56.0             44.0
5      6            14         12           2            86.0             14.0
6      7            14          9           5            64.0             36.0
7      8            15          9           6            60.0             40.0
8      9            13          7           6            54.0             46.0
9     10            14         10           4            71.0             29.0
10    11            14          9           5            64.0             36.0
11    12            16         11           5       

In [52]:
# 6

query = ('''SELECT team_name, total_wins, total_losses, spread_win, spread_loss, total_bets_for, 
                    (total_bets - total_bets_for) as total_best_against

            
            FROM(
                SELECT
                t.team_name,
                COUNT(DISTINCT CASE WHEN (t.teamid = g.homeid AND g.score_home > g.score_away) OR
                    (t.teamid = g.awayid AND g.score_home < g.score_away) THEN g.gameid END) as total_wins,
                COUNT(DISTINCT CASE WHEN (t.teamid = g.homeid AND g.score_home < g.score_away) OR
                    (t.teamid = g.awayid AND g.score_home > g.score_away) THEN g.gameid END) as total_losses,
                COUNT(DISTINCT CASE WHEN (t.teamid = g.favoriteid AND t.teamid = g.homeid AND 
                    (g.score_home + g.spread) > g.score_away) OR
                    (t.teamid = g.favoriteid AND t.teamid = g.awayid AND 
                    (g.score_away + g.spread) > g.score_home) OR
                    (g.homeid = g.favoriteid AND t.teamid = g.awayid AND 
                    (g.score_home + g.spread) < g.score_away) OR
                    (g.awayid = g.favoriteid AND t.teamid = g.homeid AND 
                    (g.score_away + g.spread) < g.score_home) THEN g.gameid END) as spread_win,
                COUNT(DISTINCT CASE WHEN (t.teamid = g.favoriteid AND t.teamid = g.homeid AND 
                    (g.score_home + g.spread) < g.score_away) OR
                    (t.teamid = g.favoriteid AND t.teamid = g.awayid AND 
                    (g.score_away + g.spread) < g.score_home) OR
                    (g.homeid = g.favoriteid AND t.teamid = g.awayid AND 
                    (g.score_home + g.spread) > g.score_away) OR
                    (g.awayid = g.favoriteid AND t.teamid = g.homeid AND 
                    (g.score_away + g.spread) > g.score_home) THEN g.gameid END) as spread_loss,
                COUNT(CASE WHEN p.bet_on = t.team_abrev THEN 1 END) AS total_bets_for, COUNT(p.gameid) AS total_bets

                FROM team t JOIN game g ON t.teamid = g.homeid OR t.teamid = g.awayid OR t.teamid = g.favoriteid
                                JOIN placed_bets p ON g.gameid = p.gameid
                WHERE g.seasonyr = 2022
                GROUP BY t.team_name) AS subquery
            ORDER BY team_name

            ''')
df = pd.read_sql(query, conn)

print(df) 

                team_name  total_wins  total_losses  spread_win  spread_loss  \
0       Arizona Cardinals           4            13           8            8   
1         Atlanta Falcons           7            10           9            8   
2         Baltimore Colts           0             0           0            0   
3        Baltimore Ravens          10             8           7            9   
4         Boston Patriots           0             0           0            0   
5           Buffalo Bills          14             4           8            9   
6       Carolina Panthers           7            10           9            8   
7           Chicago Bears           3            14           4           11   
8      Cincinnati Bengals          14             5          13            5   
9        Cleveland Browns           7            10           8            9   
10         Dallas Cowboys          13             6          10            8   
11         Denver Broncos           5   